In [15]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import numpy as np
import time
import requests
from matplotlib import pyplot as plt
import cbpro
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# Load Data

# Create a Public Client object
pc = cbpro.PublicClient()

# Create an Authenticated Client object
keys_df = pd.read_csv("cbpro_keys.txt")
passphrase = str(keys_df.iloc[0,0])
b64secret = str(keys_df.iloc[0,1])
api_key = str(keys_df.iloc[0,2])
ac = cbpro.AuthenticatedClient(api_key, b64secret, passphrase)

def get_coinbase_price(coin):
    coin = coin.upper()
    pc = cbpro.PublicClient()
    latest_cb_price = float(pc.get_product_ticker(f"{coin}-USD")['price'])
    return latest_cb_price

def get_gemini_price(coin):
    r = requests.get(f"https://api.gemini.com/v1/pubticker/{coin}usd")
    r_json = dict(r.json())
    latest_gemini_price = float(r_json['last'])
    return latest_gemini_price


def get_price_diff(coin):
    # Get Coinbase Price
    coin = coin.upper()
    pc = cbpro.PublicClient()
    latest_cb_price = float(pc.get_product_ticker(f"{coin}-USD")['price'])

    # Get Gemini Price
    r = requests.get(f"https://api.gemini.com/v1/pubticker/{coin}usd")
    r_json = dict(r.json())
    latest_gemini_price = float(r_json['last'])
    price_diff = np.round((latest_cb_price - latest_gemini_price), 4)
    return price_diff

def visualize_prices(coin, interval=60):
    coin=coin.upper

    # Get Coinbase price history (1min intervals, 300 mins)
    pc = cbpro.PublicClient()
    cb_df = pd.DataFrame(pc.get_product_historic_rates(f"{coin}-USD", granularity=interval))
    cb_df.columns = ['time', 'low', 'high', 'open', 'close', 'volume']
    cb_df['time'] = pd.to_datetime(cb_df['time'], unit='s')
    cb_df = cb_df.set_index("time")

    # Get Gemini price history (1min intervals, keep only 300 most recent data points)
    r = requests.get(f"https://api.gemini.com/v2/candles/{coin}usd/1m")
    r_json = r.json()
    gemini_df = pd.DataFrame(r_json, columns = ['time', 'open', 'high', 'low', 'close', 'volume'])
    gemini_df['time'] = pd.to_datetime(gemini_df['time'], unit='ms')
    gemini_df = gemini_df.set_index("time")
    gemini_df = gemini_df.iloc[:300, :]

    # Create plot
    fig, ax = plt.subplots()

    ax.set_xlabel("Timestamp")
    ax.set_ylabel(r"Price (USD $)")
    ax.set_title("AMP Price on Coinbase and Gemini")
    ax.grid(True)

    ax.plot(cb_df.index, cb_df.close, color='b', marker='x', label='Coinbase')
    ax.plot(gemini_df.index, gemini_df.close, color='g', marker='.', label='Gemini')
    ax.legend(loc='upper left')
    plt.show()
    
def get_price_history(coin, interval=60):
    coin=coin.upper()
    # Valid interval arguments = [60, 300, 900, 3600, 21600, 86400]
    # Get Coinbase price history
    pc = cbpro.PublicClient()
    cb_df = pd.DataFrame(pc.get_product_historic_rates(f"{coin}-USD", granularity=interval))
    cb_df.columns = ['time', 'low', 'high', 'open', 'close', 'volume']
    cb_df['time'] = pd.to_datetime(cb_df['time'], unit='s')
    #cb_df = cb_df.set_index("time")

    # Get Gemini price history
    if interval == 60:
        time_param = "1m"
    elif interval == 300:
        time_param = "5m"
    elif interval == 900:
        time_param = "15m"
    elif interval == 3600:
        time_param = "1hr"
    elif interval == 21600:
        time_param = "6hr"
    elif interval == 86400:
        time_param = "1day"
    else:
        interval = 60    
    r = requests.get(f"https://api.gemini.com/v2/candles/{coin}usd/{time_param}")
    r_json = r.json()
    gemini_df = pd.DataFrame(r_json, columns = ['time', 'open', 'high', 'low', 'close', 'volume'])
    gemini_df['time'] = pd.to_datetime(gemini_df['time'], unit='ms')
    #gemini_df = gemini_df.set_index("time")
    gemini_df = gemini_df.iloc[:300, :]    
    return cb_df, gemini_df

# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Crypto Price Dashboard"),
    html.Table([
        html.Tr([html.Td("Latest BTC Price: $"), html.Td(id='btc')]),
        html.Tr([html.Td("Latest ETH Price: $"), html.Td(id='eth')]),
        html.Tr([html.Td("Latest AMP Price: $"), html.Td(id='amp')]),
    ]),
    html.Br(),
    html.Label([
        "Select Coin",
        dcc.Dropdown(
            id='coin-dropdown', clearable=False,
            value='BTC',
            options=[
                {'label': c, 'value': c} 
                for c in ['BTC', 'ETH','AMP']
            ])
    ]),
    html.Br(),
    html.Label([
        "Select Time Period",
        dcc.RadioItems(
            id='time-interval',
            value=60,
            options=[
                {'label': "5 Hours", 'value': 60},
                {'label': "1 Day", 'value': 300},
                {'label': "3 Days", 'value': 900},
                {'label': "2 Weeks", 'value': 3600},
                {'label': "3 Months", 'value': 21600},
                {'label': "10 Months", 'value': 86400},
            ])
    ]),
    dcc.Graph(id='graph1'),
    dcc.Graph(id='graph2')
])

# Define callback to update graph
@app.callback(
    Output('btc', 'children'),
    Output('eth', 'children'),
    Output('amp', 'children'),
    Output('graph1', 'figure'),
    Output('graph2', 'figure'),
    Input("coin-dropdown", "value"),
    Input("time-interval", "value")
)

def update_figure(coin, interval):
    cb_df = get_price_history(str(coin), interval)[0]
    gemini_df = get_price_history(str(coin), interval)[1]
    btc_price = str(get_coinbase_price("btc"))
    eth_price = str(get_coinbase_price("eth"))
    amp_price = str(get_coinbase_price("amp"))
    if cb_df.iloc[0,4] > cb_df.iloc[-1,4]:
        line_color = "green"
    else:
        line_color = "red"
    return [btc_price, eth_price, amp_price,
            px.line(cb_df, x='time', y="close", render_mode="svg", title="Price History", color_discrete_sequence=[line_color, line_color]),
            px.bar(cb_df, x='time', y="volume", title="Volume History", color='close', color_continuous_scale='rdylgn')
           ]
            
# Run app and display result inline in the notebook
#app.run_server(mode='inline')
app.run_server(mode="external", host="localhost",port=8053)

Dash app running on http://localhost:8053/


In [6]:
get_price_history("amp")[0]

,time,low,high,open,close,volume
0,2021-07-15 20:33:00,0.04735,0.04744,0.04735,0.04738,107026
1,2021-07-15 20:32:00,0.04735,0.04739,0.04739,0.04735,79788
2,2021-07-15 20:31:00,0.04735,0.04741,0.04740,0.04739,18616
3,2021-07-15 20:30:00,0.04736,0.04746,0.04744,0.04736,113385
4,2021-07-15 20:29:00,0.04744,0.04746,0.04744,0.04746,33873
...,...,...,...,...,...,...
295,2021-07-15 15:38:00,0.04761,0.04785,0.04785,0.04766,289589
296,2021-07-15 15:37:00,0.04781,0.04789,0.04786,0.04789,53324
297,2021-07-15 15:36:00,0.04745,0.04788,0.04754,0.04786,1205305
298,2021-07-15 15:35:00,0.04754,0.04765,0.04759,0.04759,162794
